In [36]:
import numpy as np

In [37]:
def DMD(X, Y, tol=1e-12):
    #X - Nxm, Y - Nxm
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    return Z, Lambda


In [38]:
def DMD_exact(X, Y, tol=1e-12):
    """X - Nxm, Y - Nxm; !!Exact DMD ne vraća normirane Z[:,i]"""
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = (Y@V/Sigma)@W/Lambda
    return Z, Lambda

In [39]:
def GEDMD(X, Y, tol=1e-12, type: str=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    r = B_k@W - Z*Lambda
    return Z, Lambda, r

In [40]:
N=300
m=10
x=np.random.rand(N)
A=np.random.rand(N,N)
A = A/np.linalg.norm(A, 2)

In [41]:
X=np.empty((N,m+1))
X[:,0]=x
for i in range(m):
    X[:,i+1]=A@X[:,i]
X=X+np.random.rand(N,m+1)/N

In [42]:
Z, L = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
#Z ima stupčanu normu otprilike 1
for i in range(Z.shape[1]):
    print(np.linalg.norm(Z[:,i]))
L

1.0000000000000002
0.9999999999999994
0.9999999999999994
0.9999999999999997
0.9999999999999997
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999998
0.9999999999999998


array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.67434574+0.j        ,
       -0.51870432+0.56181541j, -0.51870432-0.56181541j])

In [43]:
Ze, Le = DMD_exact(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
Le
#isto kao L?

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.67434574+0.j        ,
       -0.51870432+0.56181541j, -0.51870432-0.56181541j])

In [44]:
Zg, Lg, rg = GEDMD(X[:,:m], X[:,1:], N*np.finfo(float).resolution)
Lg

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.51870432+0.56181541j,
       -0.51870432-0.56181541j, -0.67434574+0.j        ])

In [45]:
np.linalg.eigvals(A)[:10]

array([ 0.99895511+0.j        , -0.02983773+0.01680062j,
       -0.02983773-0.01680062j, -0.03220247+0.00853392j,
       -0.03220247-0.00853392j,  0.03447152+0.j        ,
       -0.03115523+0.00941745j, -0.03115523-0.00941745j,
        0.02655775+0.02120484j,  0.02655775-0.02120484j])

In [46]:
#!pip install pydmd

In [47]:
import pydmd
dmd1 = pydmd.DMD(svd_rank=10)
dmd1.fit(X[:,:m], X[:,1:])
dmd1.eigs

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.67434574+0.j        ,
       -0.51870432+0.56181541j, -0.51870432-0.56181541j])

In [48]:
for i in range(dmd1.eigs.shape[0]):
    print(np.linalg.norm(dmd1.reconstructed_data[:,i]-X[:,i]))

7.115796151869548e-15
2.833224828443359e-14
5.249087432562182e-14
7.673852877147148e-14
1.0240612618128462e-13
1.3128029381943235e-13
1.6220443641266865e-13
1.8585159257912513e-13
2.1239748205499281e-13
2.3746765502419185e-13


In [49]:
dmd1.reconstructed_data.shape

(300, 10)

In [50]:
#kako procijeniti s_i?
for i in range(m):
    s=np.zeros((Z.shape[0]))
    for j in range(Z.shape[1]):
        s=s+Z[:,j]*(L[j]**(i))
    print("norma razlike:",np.linalg.norm(s-X[:,i]))
    #print(s,X[:,i], sep="\n")

norma razlike: 8.380852741025778
norma razlike: 7.518788618798513
norma razlike: 7.6044936485452865
norma razlike: 7.489942515084818
norma razlike: 7.41339202255386
norma razlike: 7.465040792299442
norma razlike: 7.460869280515969
norma razlike: 7.361247530479374
norma razlike: 7.379344827998604
norma razlike: 7.359836659952813
